In [1]:
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline
['/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/colinbull/Library/Caches/pypoetry/virtualenvs/fbit-data-pipeline-aJYNke-B-py3.12/lib/python3.12/site-packages']


# Comparator computation

This computes the Euclidean distance for each establishment from a base establishment of a consistent type (Academies, Maintained schools, SEN). Each establishment is given a weight against every other establishment and the top 60 for each establishment creates the comparator set for that establishment. This is repeated for all establishments in the establishment types set and then finally across all establishments across all establishment types.

In [2]:
import time 
import numpy as np
import pandas as pd
import src.pipeline.comparator_sets as comparators
import glob 
import os

start_time = time.time()
# Create and clean directory
from pathlib import Path
Path("output/comparator-sets").mkdir(parents=True, exist_ok=True)

# files = glob.glob("output/comparator-sets/*")
# for f in files:
#     os.remove(f)

# Prepare Academy and School Data

Here we prepare the academy and maintained school data by filling in missing values in NumberOfPupils, % Free School Meals and, % Sen with the mean (at this time). 

In [3]:
academy_data = comparators.prepare_data(pd.read_parquet("output/pre-processing/academies.parquet"))
ms_data = comparators.prepare_data(pd.read_parquet("output/pre-processing/maintained_schools.parquet"))
all_schools = comparators.prepare_data(pd.read_parquet("output/pre-processing/all_schools.parquet"))

In [4]:
all_schools[~all_schools.index.isna()]

,Company Registration Number,Incorporation Date,Academy Trust UPIN,Academy UKPRN,Academy Trust Name,Academy Name,Academy UPIN,Trust Type,Date Opened,LA Name,...,URN,School Balance,School Financial Position,Partial Years Present,Did Not Submit,Teaching and Teaching support staff_Agency supply teaching staff_Per Unit,Educational supplies_Learning resources (not ICT equipment)_Per Unit,Catering staff and supplies_Income from catering_Per Unit,Other costs_Grounds maintenance_Per Unit,Other costs_Direct revenue financing_Per Unit
UKPRN,,,,,,,,,,,,,,,,,,,,,
10000001.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,100054,152780.05,Surplus,False,False,57.411738,286.469542,117.425692,6.876256,0.000000
10000044.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,124449,-363414.75,Deficit,False,False,29.118911,537.618114,0.000000,5.843418,0.000000
10000045.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,102449,199065.17,Surplus,False,False,95.570252,175.880682,149.129643,16.828328,0.000000
10000101.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,100053,-134326.20,Deficit,False,False,108.964678,470.936079,0.000000,12.009390,61.145314
10000136.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,100748,382356.54,Surplus,False,False,383.722655,346.764958,180.246773,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10092533.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,112091,26820.83,Surplus,False,False,0.000000,70.268101,145.615696,11.403165,0.000000
10092534.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,112093,-48519.58,Deficit,False,False,0.000000,152.716792,0.000000,35.000000,0.000000
10092535.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,112088,-31149.84,Deficit,False,False,175.829655,64.963448,1.197931,17.586207,0.000000


# Compute the pupil and building comparators

This creates the comparators sets across both academy and maintained schools

In [5]:
academy_comparators = comparators.compute_comparator_set(academy_data)
# print('Academy done.')
# ms_comparators = comparators.compute_comparator_set(ms_data)
# print('MS done.')
# mixed_comparators = comparators.compute_comparator_set(all_schools)

In [6]:
# ms_comparators.to_parquet("output/comparator-sets/ms_comparators.parquet")
# academy_comparators.to_parquet("output/comparator-sets/academy_comparators.parquet")
# academy_comparators.to_parquet("output/comparator-sets/mixed_comparators.parquet")
# ms_data.to_parquet("output/comparator-sets/maintained_schools.parquet")
# academy_data.to_parquet("output/comparator-sets/academies.parquet")
# all_schools.to_parquet("output/comparator-sets/all_schools.parquet")

In [7]:
academy_comparators.head(5)


,OfstedRating (name),Percentage SEN,Percentage Free school meals,Number of pupils,Total Internal Floor Area,Age Average Score,GOR (name),SchoolPhaseType,Percentage Primary Need SPLD,Percentage Primary Need MLD,Percentage Primary Need PMLD,Percentage Primary Need SEMH,Percentage Primary Need SLCN,Percentage Primary Need HI,Percentage Primary Need MSI,Percentage Primary Need PD,Percentage Primary Need ASD,Percentage Primary Need OTH,Pupils,Buildings
UKPRN,,,,,,,,,,,,,,,,,,,,
10001992,Outstanding,2.444444,24.3,885,13742.0,26.139863,Yorkshire and the Humber,Secondary,1.000000,3.000000,0.0,2.888889,3.111111,0.444444,0.000000,1.222222,1.444444,0.444444,"[10001992, 10034156, 10057549, 10067619, 10044...","[10001992, 10056094, 10044328, 10037595, 10057..."
10003498,Good,3.594771,36.1,321,15479.0,6.152077,London,Secondary,3.594771,1.960784,0.0,7.516340,3.267974,0.980392,0.000000,0.653595,4.248366,2.287582,"[10003498, 10057005, 10038743, 10047034, 10040...","[10003498, 10042036, 10064757, 10038107, 10066..."
10004147,Good,1.699346,37.5,1592,25015.0,28.648331,North East,Secondary,6.666667,4.052288,0.0,4.117647,0.588235,0.392157,0.000000,1.241830,1.111111,0.065359,"[10004147, 10036056, 10037105, 10035956, 10054...","[10004147, 10045943, 10035104, 10057641, 10039..."
10006894,Good,0.129450,23.6,1567,28298.0,23.204891,West Midlands,Secondary,0.776699,1.035599,0.0,0.776699,0.194175,0.711974,0.064725,0.517799,0.453074,0.000000,"[10006894, 10033330, 10034170, 10033567, 10034...","[10006894, 10042682, 10034154, 10027608, 10017..."
10007135,Good,3.050109,67.8,825,17240.0,16.834339,North East,Secondary,1.198257,6.862745,0.0,6.209150,0.762527,0.435730,0.000000,0.326797,1.416122,0.108932,"[10007135, 10038591, 10084234, 10063847, 10047...","[10007135, 10083667, 10042275, 10054172, 10042..."


In [8]:
import pyodbc

pyodbc.drivers()

['ODBC Driver 18 for SQL Server']

In [8]:
os.environ["DATABASE_CONNECTION_STRING"] = "Driver={ODBC Driver 18 for SQL Server};Server=localhost,1433;Database=Core;UID=sa;PWD=mystrong!Pa55word;Encrypt=no;TrustServerCertificate=yes;Connection Timeout=30"

import src.pipeline.database as db

db.insert_comparator_set("Default","Unmixed","2022", academy_comparators)